In [1]:
import pandas as pd
from numpy import datetime64
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

#from folium.features import DivIcon

pd.set_option('display.max_rows', None)

hwy_speeds = {"residential": 10, "secondary": 33, "tertiary": 60}

dir = os.getcwd()
#Do not need to change the folder name
folder_name = "ETA-Prediction"

#Change filename for different regions or datasets
filename = "eta_prediction.csv"
#df_HKG.csv is a processed file that only extract HK data from the eta-prediction.csv

path = dir[:dir.find(folder_name)]+folder_name+"-Data/1.Data/" + filename


#from folium.features import DivIcon

pd.set_option('display.max_rows', None)

df_backup = pd.read_csv(path)
df = df_backup

df.columns = ["Loc", "Vehicle", "ShipNo", "Start-Lng", "Start-Lat", "Dest-Lng", "Dest-Lat", 
    "Time-1", "Time-2", "Time-3", "Time-4", "Time-5"]

for i in range(5):
    column = "Time-"+ str(i+1)
    df[column] = pd.to_datetime(df[column], errors = 'coerce', format='%Y-%m-%d %H:%M:%S')

convert_dict = {"Loc": str, "Vehicle": str, "ShipNo": str, "Start-Lng": str, "Start-Lat": str, "Dest-Lng": str, "Dest-Lat": str,
    "Time-5": datetime64, "Time-5": datetime64, "Time-5": datetime64, "Time-5": datetime64, "Time-5": datetime64}
df = df.astype(convert_dict)

df["planDurn"] = df["Time-2"].dropna()-df["Time-1"].dropna()
df["actDurn"] = df["Time-4"].dropna()-df["Time-3"].dropna()
df["Time-4"] = df["Time-4"].fillna(df["Time-5"])
df["ActDurn-nonNull"] = (df["Time-4"].dropna()-df["Time-3"].dropna()).dt.total_seconds()/60
#df["ActDurn-nonNull"] = df["actDurn-nonNull"].dt.total_seconds()
#df["plannedRemainTime"] = df["Time-5"].dropna().astype(datetime64)-df["Time-2"].dropna().astype(datetime64)
#df["actualRemainTime"] = df["Time-5"].dropna().astype(datetime64)-df["Time-4"].dropna().astype(datetime64)

df['companyCode'] = df['ShipNo'].str[:3]
df['sysCode'] = df['ShipNo'].str[:7].str[4:]
df['countryCode'] = df['ShipNo'].str[:11].str[8:]
df['warehouse'] = df['ShipNo'].str[:16].str[12:]
df['shipNo'] = df['ShipNo'].str[-9:]



df = df.rename({"Time-1": "planStartTime", "Time-2": "planEndTime", "Time-3": "actStartTime", 
    "Time-4": "actEndTime", "Time-5": "MiscTime"}, axis = "columns")

df["origLoc"] = df[["Start-Lng", "Start-Lat"]].agg(', '.join, axis=1)
df["destLoc"] = df[["Dest-Lng", "Dest-Lat"]].agg(', '.join, axis=1)
df = df.drop(["Start-Lng", "Start-Lat", "Dest-Lng", "Dest-Lat", 'ShipNo'], axis=1)

cols = ["Loc", "Vehicle", "warehouse", "shipNo", "origLoc", "destLoc", "planStartTime", "planEndTime", "actStartTime", "actEndTime", 
    "MiscTime", "planDurn", "actDurn", "ActDurn-nonNull", "countryCode", "sysCode", "companyCode"]

df = df[cols]

df.head(5)

,Loc,Vehicle,warehouse,shipNo,origLoc,destLoc,planStartTime,planEndTime,actStartTime,actEndTime,MiscTime,planDurn,actDurn,ActDurn-nonNull,countryCode,sysCode,companyCode
0,CHN,NO-LIMIT,TSX0,000182436,"31.31611, 121.11576","30.93646, 121.46803",2017-02-07,2017-02-07 01:01:28,2019-01-16 22:04:04,2017-02-08 23:59:00,2017-02-08 23:59:00,0 days 01:01:28,NaT,-1.017965e+06,CHN,TMS,LFL
1,CHN,NO-LIMIT,TSX0,000182584,"31.31611, 121.11576","30.93646, 121.46803",2017-02-07,2017-02-07 01:01:28,2019-01-16 22:04:04,2017-02-08 23:59:00,2017-02-08 23:59:00,0 days 01:01:28,NaT,-1.017965e+06,CHN,TMS,LFL
2,CHN,NO-LIMIT,TSX0,000183341,"31.31611, 121.11576","30.93646, 121.46803",2017-02-07,2017-02-07 01:01:28,2019-01-16 22:04:04,2017-02-08 23:59:00,2017-02-08 23:59:00,0 days 01:01:28,NaT,-1.017965e+06,CHN,TMS,LFL
3,CHN,NO-LIMIT,TSX0,000183345,"31.31611, 121.11576","30.93646, 121.46803",2017-02-07,2017-02-07 01:01:28,2019-01-16 22:04:04,2017-02-08 23:59:00,2017-02-08 23:59:00,0 days 01:01:28,NaT,-1.017965e+06,CHN,TMS,LFL
4,CHN,NO-LIMIT,TSX0,000183763,"31.31611, 121.11576","30.93646, 121.46803",2017-02-07,2017-02-07 01:01:28,2019-01-16 22:04:04,2017-02-08 23:59:00,2017-02-08 23:59:00,0 days 01:01:28,NaT,-1.017965e+06,CHN,TMS,LFL
